<a href="https://colab.research.google.com/github/armstrohiro/Pose-Estimations/blob/main/HRNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sed -i 's/opencv-python==3.4.1.15/opencv-python==4.8.1.78/' requirements.txt

In [ ]:
!pip install opencv-python

In [ ]:
!pip install opencv-python-headless

In [ ]:
!git clone https://github.com/leoxiaobin/deep-high-resolution-net.pytorch.git

fatal: destination path 'deep-high-resolution-net.pytorch' already exists and is not an empty directory.


In [ ]:
# 二重・三重のディレクトリを削除
%rm -rf /content/mmpose/deep-high-resolution-net.pytorch/deep-high-resolution-net.pytorch

In [ ]:
%cd /content/mmpose/deep-high-resolution-net.pytorch

/content/mmpose/deep-high-resolution-net.pytorch


In [ ]:
# 古いリポジトリを削除
%rm -rf /content/mmpose/deep-high-resolution-net.pytorch

# 正しい場所に再クローン
!git clone https://github.com/leoxiaobin/deep-high-resolution-net.pytorch.git /content/mmpose/deep-high-resolution-net.pytorch

# クローン後に再確認
%ls /content/mmpose/deep-high-resolution-net.pytorch/lib

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
Cloning into '/content/mmpose/deep-high-resolution-net.pytorch'...
fatal: Unable to read current working directory: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
ls: cannot access '/content/mmpose/deep-high-resolution-net.pytorch/lib': No such file or directory


In [ ]:
%cd lib
!python setup.py build_ext --inplace
%cd ..

/content/mmpose/deep-high-resolution-net.pytorch/lib
python3: can't open file '/content/mmpose/deep-high-resolution-net.pytorch/lib/setup.py': [Errno 2] No such file or directory
/content/mmpose/deep-high-resolution-net.pytorch


In [ ]:
import os
import pandas as pd
import cv2
from mmpose.apis import init_pose_model, inference_topdown

# 設定
pose_config = '/content/top_down/hrnet/hrnet_w48_coco_256x192.py'
pose_checkpoint = '/content/checkpoints/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth'
video_folder = '/content/output/'  # 動画ファイルの保存場所
output_csv_folder = '/content/outputs/csv/'  # 結果を保存するフォルダ
os.makedirs(output_csv_folder, exist_ok=True)

# モデルを初期化
pose_model = init_pose_model(pose_config, pose_checkpoint, device='cuda')  # 'cuda' または 'cpu'

# 動画ファイルを処理
for video_name in os.listdir(video_folder):
    if not video_name.endswith('.mp4'):  # MP4ファイルのみ処理
        continue

    video_path = os.path.join(video_folder, video_name)
    output_csv_path = os.path.join(output_csv_folder, f'{os.path.splitext(video_name)[0]}.csv')
    print(f'Processing video: {video_name}')

    # 動画を読み込む
    cap = cv2.VideoCapture(video_path)
    frame_idx = 0
    results = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # フレームサイズ取得
        h, w, _ = frame.shape
        bbox = [[0, 0, w, h, 1.0]]  # フレーム全体を対象とするバウンディングボックス

        # 推論
        pose_results = inference_topdown(pose_model, frame, bbox)

        # 結果を保存
        for person in pose_results:
            keypoints = person['keypoints']
            row = [frame_idx] + keypoints.flatten().tolist()
            results.append(row)

        frame_idx += 1

    cap.release()

    # データをCSVに保存
    columns = ['frame'] + [f'kp_{i}_{coord}' for i in range(len(keypoints)) for coord in ['x', 'y', 'score']]
    df = pd.DataFrame(results, columns=columns)
    df.to_csv(output_csv_path, index=False)
    print(f'Saved results to {output_csv_path}')

print('All videos processed.')


KeyError: 'ASGD is already registered in optimizer at torch.optim.asgd'

In [ ]:
from mmengine.registry import Registry

OPTIMIZERS = Registry('optimizer')

12/11 03:43:34 - mmengine - WARNING - set scope as "mmengine" when scope can not be inferred. You can silence this warning by passing a "scope" argument to Registry like `Registry(name, scope="toy")`


In [ ]:
import os
import cv2

# 入力フォルダと出力フォルダ
frames_folder = '/content/frames'  # フレーム保存フォルダ
predictions_folder = '/content/predictions'  # 推論結果保存フォルダ
os.makedirs(predictions_folder, exist_ok=True)

# フレームを個別に処理
for video_folder in sorted(os.listdir(frames_folder)):
    video_frames_path = os.path.join(frames_folder, video_folder)
    video_predictions_path = os.path.join(predictions_folder, video_folder)
    os.makedirs(video_predictions_path, exist_ok=True)

    print(f"Processing frames in {video_folder}...")

    for frame_file in sorted(os.listdir(video_frames_path)):
        if not frame_file.endswith('.jpg'):
            continue

        frame_path = os.path.join(video_frames_path, frame_file)
        output_path = os.path.join(video_predictions_path, frame_file.replace('.jpg', '_result.jpg'))

        # 推論実行
        !python demo/topdown_demo.py \
            --pose-model hrnet_w48_coco_384x288 \
            --device cuda \
            --save {output_path} \
            {frame_path}

        print(f"Processed: {frame_file}")

Processing frames in 00001...
Traceback (most recent call last):
  File "/content/demo/topdown_demo.py", line 40, in <module>
    model = ep.TopDown(pose_model_name=args.pose_model,
NameError: name 'ep' is not defined
Processed: frame_0000.jpg
Traceback (most recent call last):
  File "/content/demo/topdown_demo.py", line 40, in <module>
    model = ep.TopDown(pose_model_name=args.pose_model,
NameError: name 'ep' is not defined
Processed: frame_0001.jpg
Traceback (most recent call last):
  File "/content/demo/topdown_demo.py", line 40, in <module>
    model = ep.TopDown(pose_model_name=args.pose_model,
NameError: name 'ep' is not defined
Processed: frame_0002.jpg
Traceback (most recent call last):
  File "/content/demo/topdown_demo.py", line 40, in <module>
    model = ep.TopDown(pose_model_name=args.pose_model,
NameError: name 'ep' is not defined
Processed: frame_0003.jpg
Traceback (most recent call last):
  File "/content/demo/topdown_demo.py", line 40, in <module>
    model = ep.Top

In [ ]:
new_code = """
import argparse
import cv2
from mmpose.apis import (inference_topdown, init_pose_model, vis_pose_result)

def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('img', help='Path to the input image file.')
    parser.add_argument('--pose-config', type=str, default='configs/top_down/hrnet_w48_coco_256x192.py',
                        help='Path to the pose model config file.')
    parser.add_argument('--pose-checkpoint', type=str, default='checkpoints/hrnet_w48_coco_256x192.pth',
                        help='Path to the pose model checkpoint file.')
    parser.add_argument('--device', type=str, default='cuda', help='Device used for inference (cuda or cpu).')
    parser.add_argument('--show', action='store_true', help='Whether to display the result.')
    parser.add_argument('--save', type=str, default='', help='Path to save the output image.')
    return parser.parse_args()

if __name__ == '__main__':
    args = get_args()

    # モデルを初期化
    pose_model = init_pose_model(args.pose_config, args.pose_checkpoint, device=args.device)

    # 入力画像を読み込む
    image = cv2.imread(args.img)
    if image is None:
        raise FileNotFoundError(f'Image file not found: {args.img}')

    # 仮のバウンディングボックス（全体を対象にする）
    h, w, _ = image.shape
    bbox = [[0, 0, w, h, 1.0]]  # x, y, width, height, score

    # 推論を実行
    pose_results = inference_topdown(pose_model, image, bbox)

    # 結果を描画
    vis_image = vis_pose_result(pose_model, image, pose_results, dataset='TopDownCocoDataset', show=args.show)

    # 保存
    if args.save:
        cv2.imwrite(args.save, vis_image)

"""

# 修正内容を書き込み
file_path = '/content/demo/topdown_demo.py'
with open(file_path, 'w') as f:
    f.write(new_code)

print(f"Updated {file_path}")

Updated /content/demo/topdown_demo.py


In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 94.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
!pip uninstall -y torch torchvision torchaudio

Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121


In [ ]:
!pip install torchaudio

In [ ]:
import argparse
import cv2
from mmpose.apis import (inference_topdown, init_pose_model, vis_pose_result)

def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('img', help='Path to the input image file.')
    parser.add_argument('--pose-config', type=str, default='configs/top_down/hrnet_w48_coco_256x192.py',
                        help='Path to the pose model config file.')
    parser.add_argument('--pose-checkpoint', type=str, default='checkpoints/hrnet_w48_coco_256x192.pth',
                        help='Path to the pose model checkpoint file.')
    parser.add_argument('--device', type=str, default='cuda', help='Device used for inference (cuda or cpu).')
    parser.add_argument('--show', action='store_true', help='Whether to display the result.')
    parser.add_argument('--save', type=str, default='', help='Path to save the output image.')
    return parser.parse_args()

if __name__ == '__main__':
    args = get_args()

    # モデルを初期化
    pose_model = init_pose_model(args.pose_config, args.pose_checkpoint, device=args.device)

    # 入力画像を読み込む
    image = cv2.imread(args.img)
    if image is None:
        raise FileNotFoundError(f'Image file not found: {args.img}')

    # 仮のバウンディングボックス（全体を対象にする）
    h, w, _ = image.shape
    bbox = [[0, 0, w, h, 1.0]]  # x, y, width, height, score

    # 推論を実行
    pose_results = inference_topdown(pose_model, image, bbox)

    # 結果を描画
    vis_image = vis_pose_result(pose_model, image, pose_results, dataset='TopDownCocoDataset', show=args.show)

    # 保存
    if args.save:
        cv2.imwrite(args.save, vis_image)

KeyError: 'ASGD is already registered in optimizer at torch.optim.asgd'

In [ ]:
!rm -rf /content/mmpose  # 古いディレクトリを削除
!git clone https://github.com/open-mmlab/mmpose.git /content/mmpose
%cd /content/mmpose

Cloning into '/content/mmpose'...
remote: Enumerating objects: 31261, done.
remote: Counting objects: 100% (363/363), done.
remote: Compressing objects: 100% (234/234), done.
remote: Total 31261 (delta 174), reused 266 (delta 127), pack-reused 30898 (from 1)
Receiving objects: 100% (31261/31261), 54.31 MiB | 33.70 MiB/s, done.
Resolving deltas: 100% (22001/22001), done.
/content/mmpose


In [ ]:
!pip install -e .

Obtaining file:///content/mmpose
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/jin-s13/xtcocoapi to /tmp/pip-install-ta3b7ueb/xtcocotools_2f1e2847ee8b495497964ca4e9bb4a01
  Running command git clone --filter=blob:none --quiet https://github.com/jin-s13/xtcocoapi /tmp/pip-install-ta3b7ueb/xtcocotools_2f1e2847ee8b495497964ca4e9bb4a01
  Resolved https://github.com/jin-s13/xtcocoapi to commit d74033ff1635e9002133b2380862bc2b728584d2
  Preparing metadata (setup.py) ... done
  Running setup.py develop for mmpose


In [ ]:
!ls /content/mmpose/demo

body3d_pose_lifter_demo.py  image_demo.py	   mmtracking_cfg
bottomup_demo.py	    inferencer_demo.py	   resources
docs			    mmdetection_cfg	   topdown_demo_with_mmdet.py
hand3d_internet_demo.py     MMPose_Tutorial.ipynb


In [ ]:
!mim install mmdet

Looking in links: https://download.openmmlab.com/mmcv/dist/cu121/torch2.5.0/index.html
Ignoring mmcv: markers 'extra == "mim"' don't match your environment
Ignoring mmengine: markers 'extra == "mim"' don't match your environment


In [ ]:
# MMEngine, MMCV, MMDetection, MMPoseのバージョン確認
!pip show mmengine mmcv mmdet mmpose

Name: mmengine
Version: 0.10.5
Summary: Engine of OpenMMLab projects
Home-page: https://github.com/open-mmlab/mmengine
Author: MMEngine Authors
Author-email: openmmlab@gmail.com
License: UNKNOWN
Location: /usr/local/lib/python3.10/dist-packages
Requires: addict, matplotlib, numpy, opencv-python, pyyaml, rich, termcolor, yapf
Required-by: mmcv
---
Name: mmcv
Version: 2.2.0
Summary: OpenMMLab Computer Vision Foundation
Home-page: https://github.com/open-mmlab/mmcv
Author: MMCV Contributors
Author-email: openmmlab@gmail.com
License: UNKNOWN
Location: /usr/local/lib/python3.10/dist-packages
Requires: addict, mmengine, numpy, packaging, Pillow, pyyaml, yapf
Required-by: 
---
Name: mmdet
Version: 3.3.0
Summary: OpenMMLab Detection Toolbox and Benchmark
Home-page: https://github.com/open-mmlab/mmdetection
Author: MMDetection Contributors
Author-email: openmmlab@gmail.com
License: Apache License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: matplotlib, numpy, pycocotools, sci

In [ ]:
!wget -P checkpoints https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth

--2024-12-11 03:10:44--  https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 8.25.82.179, 8.25.82.163, 8.25.82.180, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|8.25.82.179|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167287506 (160M) [application/octet-stream]
Saving to: ‘checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth’

faster_rcnn_r50_fpn 100%[===================>] 159.54M  5.71MB/s    in 37s     

2024-12-11 03:11:22 (4.28 MB/s) - ‘checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth’ saved [167287506/167287506]



In [ ]:
!python demo/topdown_demo_with_mmdet.py \
    configs/top_down/hrnet_w48_coco_256x192.py \
    checkpoints/hrnet_w48_coco_256x192.pth \
    --input /content/frames \
    --output /content/predictions \
    --det-config mmdetection_cfg/faster_rcnn_r50_fpn_coco.py \
    --det-checkpoint checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth \
    --device cuda

Traceback (most recent call last):
  File "/content/demo/topdown_demo_with_mmdet.py", line 15, in <module>
    from mmpose.apis import inference_topdown
  File "/content/mmpose/mmpose/apis/__init__.py", line 2, in <module>
    from .inference import (collect_multi_frames, inference_bottomup,
  File "/content/mmpose/mmpose/apis/inference.py", line 11, in <module>
    from mmengine.model.utils import revert_sync_batchnorm
  File "/usr/local/lib/python3.10/dist-packages/mmengine/model/__init__.py", line 6, in <module>
    from .base_model import BaseDataPreprocessor, BaseModel, ImgDataPreprocessor
  File "/usr/local/lib/python3.10/dist-packages/mmengine/model/base_model/__init__.py", line 2, in <module>
    from .base_model import BaseModel
  File "/usr/local/lib/python3.10/dist-packages/mmengine/model/base_model/base_model.py", line 9, in <module>
    from mmengine.optim import OptimWrapper
  File "/usr/local/lib/python3.10/dist-packages/mmengine/optim/__init__.py", line 2, in <module>
 

In [ ]:
# PyTorchと関連ライブラリをアンインストール
!pip uninstall -y torch torchvision torchaudio

# 安定版のPyTorchをインストール（例: 2.0.1）
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2+cu121 -f https://download.pytorch.org/whl/cu121/torch_stable.html

Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121
Looking in links: https://download.pytorch.org/whl/cu121/torch_stable.html
ERROR: Ignored the following yanked versions: 2.0.0
ERROR: Could not find a version that satisfies the requirement torchaudio==2.0.2+cu121 (from versions: 0.11.0, 0.12.0, 0.12.1, 0.13.0, 0.13.1, 2.0.1, 2.0.2, 2.1.0, 2.1.0+cu121, 2.1.1, 2.1.1+cu121, 2.1.2, 2.1.2+cu121, 2.2.0, 2.2.0+cu121, 2.2.1, 2.2.1+cu121, 2.2.2, 2.2.2+cu121, 2.3.0, 2.3.0+cu121, 2.3.1, 2.3.1+cu121, 2.4.0, 2.4.1, 2.5.0, 2.5.1)
ERROR: No matching distribution found for torchaudio==2.0.2+cu121
